In [ ]:
import openai
import random
from tqdm import tqdm
from pprint import pprint
import pandas as pd
import time as time
import pickle
openai.api_key = ""

In [ ]:
def extract(lang, num_samples):
    file_path = f'harmful_instructions/harmful_instructions_{lang}.txt'
    lines = []
    with open(file_path, 'r') as file:
        for line in file:
            lines.append(line.strip() + '\n\n')

    random.seed(10)
    random.shuffle(lines)
    return lines[:num_samples]

In [ ]:
LANGS = ['en', 'ml', 'ja', 'hu', 'sw']
NUM_SAMPLES = 100

In [ ]:
files = {lang : extract(lang, NUM_SAMPLES) for lang in LANGS}

In [ ]:
BATCH_SIZE = 20
outputs = {}
for lang in LANGS:
    outputs[lang] = []
    for i in tqdm(range(0, len(files[lang]), BATCH_SIZE)):
        batch_instructions = files[lang][i:i + BATCH_SIZE]
        responses = openai.Completion.create(
            model = "text-davinci-003",
            temperature = 0.0,
            max_tokens = 512,
            prompt = batch_instructions
        )

        for response in responses.choices:
            output_text = response.text
            outputs[lang].append(output_text)

In [ ]:
from googletrans import Translator
translator = Translator()

def translate(input_text, src, dest):
    max_retries = 5
    retry_count = 0
    translated_text = None

    while retry_count < max_retries:
        try:
            translated_text = translator.translate(input_text, src=src, dest=dest).text
            break
        except Exception as e:
            print(f"Translation attempt {retry_count + 1} failed with error: {e}")
            retry_count += 1
            time.sleep(3)

    if translated_text is not None:
        return translated_text

    else:
        print('failed unfortunately')
        assert False

In [ ]:
outputs_en = {}
for lang in LANGS:
    if lang == 'en':
        outputs_en['en'] = outputs['en']
    else:
        outputs_en[lang] = []
        for output in tqdm(outputs[lang]):
            outputs_en[lang].append(translate(output, lang, 'en'))

In [ ]:
for lang in LANGS:
    my_list = outputs_en[lang]
    file_path = f"harmful_dumps/temp_{lang}.txt"

    with open(file_path, 'w') as file:
        for string in my_list:
            file.write(repr(string) + '\n')

In [ ]:
with open('harmful_dumps/text-davinci-003.pkl', 'wb') as file:
    pickle.dump([files, outputs, outputs_en], file)

In [ ]:
with open('harmful_dumps/text-davinci-003.pkl', 'rb') as file:
    unpickled = pickle.load(file)